In [ ]:
## Create the last year's MTD and YoY metrics program

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F
from pyspark.sql.functions import format_string

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/sharb24/SBB/thural2/record.py').read())


In [ ]:
from datetime import datetime
last_year0 = datetime.strptime(rec_date, '%Y%m%d').strftime('%m/%d/%Y')

## add 1 day
from datetime import datetime, timedelta
last_day00=pd.to_datetime(last_year0) + timedelta(days=1)
print(last_day00)

## subtract 1 year
from datetime import datetime, timedelta
last_year=pd.to_datetime(last_day00) - timedelta(days=1*365)
print(last_year)

## convert to old scorecard format
from datetime import datetime
last_yearf = last_year.strftime('%m/%d/%Y')
print(last_yearf)


In [ ]:
######## IMPORT FUNCTIONS

## CREATE THE previous year's MTD, QTD, AND YTD VARIABLES
exec("""def yoyvarfunc(work): 
    work['prev_november'] = ((work.november.fillna(0)) - (work.last_november.fillna(0)))/(work.november.fillna(0))
    work['prev_december'] = ((work.december.fillna(0)) - (work.last_december.fillna(0)))/(work.december.fillna(0))
    work['prev_january'] = ((work.january.fillna(0)) - (work.last_january.fillna(0)))/(work.january.fillna(0))
    work['prev_february'] = ((work.february.fillna(0)) - (work.last_february.fillna(0)))/(work.february.fillna(0))
    work['prev_march'] = ((work.march.fillna(0)) - (work.last_march.fillna(0)))/(work.march.fillna(0))
    work['prev_april'] = ((work.april.fillna(0)) - (work.last_april.fillna(0)))/(work.april.fillna(0))
    work['prev_may'] = ((work.may.fillna(0)) - (work.last_may.fillna(0)))/(work.may.fillna(0))
    work['prev_june'] = ((work.june.fillna(0)) - (work.last_june.fillna(0)))/(work.june.fillna(0))
    work['prev_july'] = ((work.july.fillna(0)) - (work.last_july.fillna(0)))/(work.july.fillna(0))
    work['prev_august'] = ((work.august.fillna(0)) - (work.last_august.fillna(0)))/(work.august.fillna(0))
    work['prev_september'] = ((work.september.fillna(0)) - (work.last_september.fillna(0)))/(work.september.fillna(0))
    work['prev_october'] = ((work.october.fillna(0)) - (work.last_october.fillna(0)))/(work.october.fillna(0))
    work['Q1_YoY'] = ((work.q1.fillna(0)) - (work.last_q1.fillna(0)))/(work.q1.fillna(0))
    work['Q2_YoY'] = ((work.q2.fillna(0)) - (work.last_q2.fillna(0)))/(work.q2.fillna(0))
    work['Q3_YoY'] = ((work.q3.fillna(0)) - (work.last_q3.fillna(0)))/(work.q3.fillna(0))
    work['Q4_YoY'] = ((work.q4.fillna(0)) - (work.last_q4.fillna(0)))/(work.q4.fillna(0))
    work['YTD_YoY'] = ((work.ytd.fillna(0)) - (work.last_ytd.fillna(0)))/(work.ytd.fillna(0))
    """)

total_onetd1=('Total Wealth Referrals','AMSB Retail Referrals','Total TD Securities Direct Trade','Total OneTD Widgets')
total_be1=('Business Credit Protection(BCP) Widgets','Total Merchant Solutions','Total Cash Management Services (including RDC)',
         'Total Credit Cards','Total Business Essentials Widgets')

In [ ]:
####################### READ IN THE CURRENT AM DATA METRICS (EXCEPT LEI) #######################################
df_metrics= spark.table('anp_cabbtdct1_final.SBB_SC_OUTPUT_ALL')
df_metrics1=df_metrics.toPandas()

## get the AMSB data
df_metric_amsb=df_metrics1[(df_metrics1.level=='AMSB') & (df_metrics1.record_date==rec_date)]

## get the AM data
df_metric_am=df_metrics1[(df_metrics1.level=='AM') & (df_metrics1.record_date==rec_date)]

## get the SM data
df_metric_sm=df_metrics1[(df_metrics1.level=='SM') & (df_metrics1.record_date==rec_date)]


In [ ]:
df_metric_sm[(df_metric_sm.employee_name=="GTA") & (df_metric_sm.metric_name=="AMSB Retail Referrals")]

In [ ]:

################################### GET ALL AMSBs #################################################

## get all AMSBs from HR data
df=spark.sql("select acf2_id,employee_name,am_cost_center,am_cost_center_name,am_cost_center_full_name, \
                     sm_cost_center,sm_cost_center_name from \
                     (select *, row_number() over(partition by acf2_id order by record_date desc) as ran \
                     from anp_cabbtdct1_final.HRM_SBB_ALIGNMENT0)b where ran=1")
sbb_hr1=df.toPandas()

sbb_hr2=sbb_hr1.copy()
sbb_hr2['level']='AMSB'
sbb_hr2['scorecard_filter']='SBB'
sbb_hr3=sbb_hr2[['acf2_id','employee_name','level','scorecard_filter','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                 'sm_cost_center','sm_cost_center_name']]

## get unique metric_name and unique benchmarks
df_metrics1_all3=df_metric_amsb[['level','metric_name','sm_cost_center','q1_benchmark','q2_benchmark','q3_benchmark','q4_benchmark','ytd_benchmark']].drop_duplicates()

## merge HR data with all metric names and benchmarks
result_amsb18 = pd.merge(sbb_hr3,df_metrics1_all3,on=['level','sm_cost_center'])

## now merge with data so each AMSB has each metric name
    ##drop columns before merge
all_metrics_final_amsb1=df_metric_amsb.drop(columns=['employee_name','q1_benchmark','q2_benchmark','q3_benchmark','q4_benchmark','ytd_benchmark',
                                                            'am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center',
                                                             'sm_cost_center_name','scorecard_filter'])
result_amsb28 = pd.merge(result_amsb18,all_metrics_final_amsb1,on=['acf2_id','level','metric_name'],how='left')

result_amsb28['record_date']=dteq

result_amsb28['prev_november']=0
result_amsb28['prev_december']=0
result_amsb28['prev_january']=0
result_amsb28['prev_february']=0
result_amsb28['prev_march']=0
result_amsb28['prev_april']=0
result_amsb28['prev_may']=0
result_amsb28['prev_june']=0
result_amsb28['prev_july']=0
result_amsb28['prev_august']=0
result_amsb28['prev_september']=0
result_amsb28['prev_october']=0
result_amsb28['Q1_YoY']=0
result_amsb28['Q2_YoY']=0
result_amsb28['Q3_YoY']=0
result_amsb28['Q4_YoY']=0
result_amsb28['YTD_YoY']=0

#### get relevant vars
df_amsb_yoy=result_amsb28[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','Q1_YoY','q2','q2_benchmark','Q2_YoY',
                            'q3','q3_benchmark','Q3_YoY','q4','q4_benchmark','Q4_YoY','ytd','ytd_benchmark','YTD_YoY',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]



In [ ]:
#df_metrics1[(df_metrics1.acf2_id=='ALEXAT2') & (df_metrics1.metric_name=='ALL Total Commercial Volume')]

In [ ]:

################################################ get all AMs #######################################

#### hr data
sbb_hr7=sbb_hr1.copy()
sbb_hr31=sbb_hr7[['am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center','sm_cost_center_name']].drop_duplicates()
sbb_hr31['level']='AM'

sc_list=('SBB','Retail','Total')
## create dfs
df_am = pd.DataFrame({'scorecard_filter':sc_list})
df_am['level']='AM'

## every AM CC has a scorecard_filter
result67 = pd.merge(df_am,sbb_hr31,on=['level'])


## get unique metric_name and unique benchmarks
df_metrics1_all39=df_metric_am[['metric_name','scorecard_filter','q1_benchmark','q2_benchmark','q3_benchmark','q4_benchmark','ytd_benchmark']].drop_duplicates()
df_metrics1_all39['level']='AM'

## merge HR data with all metric names and benchmarks
result_am188 = pd.merge(result67,df_metrics1_all39,on=['level','scorecard_filter'])

## now merge with data so each AM has each metric name
    ##drop columns before merge
all_metrics_final_am1=df_metric_am.drop(columns=['employee_name','q1_benchmark','q2_benchmark','q3_benchmark','q4_benchmark','ytd_benchmark',
                                                        'am_cost_center_name','am_cost_center_full_name','sm_cost_center',
                                                        'sm_cost_center_name'])
result_am288 = pd.merge(result_am188,all_metrics_final_am1,on=['am_cost_center','scorecard_filter','level','metric_name'],how='left')

result_am288['employee_name']=result_am288.am_cost_center_full_name
result_am288['acf2_id']=''
result_am288['record_date']=dteq

## keep relevant vars
result_am_all=result_am288[['metric_name','level','acf2_id','employee_name',
                            'november','december','january','february','march','april','may',
                            'june','july','august','september','october','q1','q1_benchmark',
                            'q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
                            'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
result_am_all[(result_am_all.am_cost_center=='2187') & (result_am_all.scorecard_filter=='SBB')].shape[0]

In [ ]:

############################################ get SMs ##############################################################

#### hr data
sbb_hr71=sbb_hr1.copy()
sbb_hr311=sbb_hr71[['sm_cost_center','sm_cost_center_name']].drop_duplicates()
sbb_hr311['level']='SM'

sc_list=('SBB','Retail','Total')
## create dfs
df_sm = pd.DataFrame({'scorecard_filter':sc_list})
df_sm['level']='SM'

## every SM CC has a scorecard_filter
result677 = pd.merge(df_sm,sbb_hr311,on=['level'])


## get unique metric_name and unique benchmarks
df_metrics1_all399=df_metric_sm[['metric_name','scorecard_filter','q1_benchmark','q2_benchmark','q3_benchmark','q4_benchmark','ytd_benchmark']].drop_duplicates()
df_metrics1_all399['level']='SM'

## merge HR data with all metric names and benchmarks
result_sm188 = pd.merge(result677,df_metrics1_all399,on=['level','scorecard_filter'])

## now merge with data so each SM has each metric name
    ##drop columns before merge
all_metrics_final_sm1=df_metric_sm.drop(columns=['employee_name','q1_benchmark','q2_benchmark','q3_benchmark','q4_benchmark','ytd_benchmark',
                                                'am_cost_center','am_cost_center_name','am_cost_center_full_name','sm_cost_center_name'])
result_sm288 = pd.merge(result_sm188,all_metrics_final_sm1,on=['sm_cost_center','scorecard_filter','level','metric_name'],how='left')

result_sm288['am_cost_center']=''
result_sm288['am_cost_center_name']=''
result_sm288['am_cost_center_full_name']=''
result_sm288['employee_name']=result_sm288.sm_cost_center_name
result_sm288['acf2_id']=''
result_sm288['record_date']=dteq

## keep relevant vars
result_sm_all=result_sm288[['metric_name','level','acf2_id','employee_name',
                            'november','december','january','february','march','april','may',
                            'june','july','august','september','october','q1','q1_benchmark',
                            'q2','q2_benchmark','q3','q3_benchmark','q4','q4_benchmark',
                            'ytd','ytd_benchmark','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
result_sm_all[(result_sm_all.sm_cost_center=='2043') & (result_sm_all.scorecard_filter=='SBB')].metric_name.unique() ## missing TDEF volume

In [ ]:
##################################################################################################
############################# get the previous year's MTD and YTD data ###########################

############ get historical AM data ##############
dfamhist= spark.table('anp_cabbtdct1_working.AM_FY22_SCORECARD_HISTORY1')
dfamhist1=dfamhist.toPandas()
dfamhist1.columns=dfamhist1.columns.str.lower()
## get last year's weekly data (mm/dd/yyyy) varchar
dfamhist2=dfamhist1[(dfamhist1.scorecard_date==last_yearf)]

##change vars to float
dfamhist2= dfamhist2.astype({"last_november": float,"last_december": float, "last_january": float,
                            "last_february": float, "last_march": float, "last_april": float,
                            "last_may": float, "last_june": float, "last_july": float, "last_august": float,
                            "last_september": float, "last_october": float, "last_q1": float, "last_q2": float,
                            "last_q3": float, "last_q4": float, "last_ytd": float})

############ get historical SM data #############
dfsmhist= spark.table('anp_cabbtdct1_working.SM_FY22_SCORECARD_HISTORY1')
dfsmhist1=dfsmhist.toPandas()
dfsmhist1.columns=dfsmhist1.columns.str.lower()
## get last year's weekly data (mm/dd/yyyy) varchar
dfsmhist2=dfsmhist1[(dfsmhist1.scorecard_date==last_yearf)]

##change vars to float
dfsmhist2= dfsmhist2.astype({"last_november": float,"last_december": float, "last_january": float,
                            "last_february": float, "last_march": float, "last_april": float,
                            "last_may": float, "last_june": float, "last_july": float, "last_august": float,
                            "last_september": float, "last_october": float, "last_q1": float, "last_q2": float,
                            "last_q3": float, "last_q4": float, "last_ytd": float})

##rename region_id
dfsmhist2.rename(columns={"region_id":'sm_cost_center'},inplace=True)


In [ ]:
####################### CALCULATE AM's YoY MTD AND YTD METRICS #############################################

## merge the current data to last year's data
am_metric_calc=pd.merge(result_am_all,dfamhist2,on=['metric_name','scorecard_filter','am_cost_center'],how="left")

## MTD,QTD,YTD calculations
yoyvarfunc(am_metric_calc)

#### get relevant vars
df_am_yoy=am_metric_calc[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','Q1_YoY','q2','q2_benchmark','Q2_YoY',
                            'q3','q3_benchmark','Q3_YoY','q4','q4_benchmark','Q4_YoY','ytd','ytd_benchmark','YTD_YoY',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
#df_am_yoy.head(5)

In [ ]:
dfsmhist2.metric_name.unique() ##### TD SECURITIES AND LEI ARE MISSING FROM HISTORY

In [ ]:
####################### CALCULATE SM's YoY MTD AND YTD METRICS #############################################

## merge the current data to last year's data
sm_metric_calc=pd.merge(result_sm_all,dfsmhist2,on=['metric_name','scorecard_filter','sm_cost_center'],how="left")

## MTD,QTD,YTD calculations
yoyvarfunc(sm_metric_calc)

#### get relevant vars
df_sm_yoy=sm_metric_calc[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','Q1_YoY','q2','q2_benchmark','Q2_YoY',
                            'q3','q3_benchmark','Q3_YoY','q4','q4_benchmark','Q4_YoY','ytd','ytd_benchmark','YTD_YoY',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
df_sm_yoy.head(5)

In [ ]:
################################# CALCULATE FOR NO (roll up SM history to get total widgets) ################################################

###### get historical OneTD widgets (sum of Retail Referrals, wealth referrals, td securities)
dfsmhist21=dfsmhist2[dfsmhist2.metric_name.isin(total_onetd1)]

## order vars
dfsmhist23=dfsmhist21[['sm_cost_center','metric_name','scorecard_filter','scorecard_date','last_november','last_december','last_january',
                      'last_february','last_march','last_april','last_may','last_june','last_july','last_august','last_september',
                      'last_october','last_q1','last_q2','last_q3','last_q4','last_ytd']]


###### get historical Business Essentials widgets (sum of bcp, credit cards, merchant, cms)
dfsmhist26=dfsmhist2[dfsmhist2.metric_name.isin(total_be1)]

## order vars
dfsmhist27=dfsmhist26[['sm_cost_center','metric_name','scorecard_filter','scorecard_date','last_november','last_december','last_january',
                      'last_february','last_march','last_april','last_may','last_june','last_july','last_august','last_september',
                      'last_october','last_q1','last_q2','last_q3','last_q4','last_ytd']]


#### concatenate to rest of dataframe
df_sm_all=pd.concat([dfsmhist27,dfsmhist23,dfsmhist2],axis=0,sort=False)


##rename legacy metrics to match current data
df_sm_all.rename(columns = {"Total TD Securities Direct Trade":'TD Securities Direct Trade'}, inplace = True)

In [ ]:
#df_sm_all

In [ ]:
##################### CALCULATE NATIONAL OFFICE LAST YEAR
##historical data
df_nso_all27=df_sm_all.groupby(by=['scorecard_filter','metric_name','scorecard_date'],as_index=False).aggregate({
                        'last_november': 'sum',
                        'last_december': 'sum',
                        'last_january': 'sum',
                        'last_february': 'sum',
                        'last_march': 'sum',
                        'last_april': 'sum',
                        'last_may': 'sum',
                        'last_june': 'sum',
                        'last_july': 'sum',
                        'last_august': 'sum',
                        'last_september': 'sum',
                        'last_october': 'sum',
                        'last_q1': 'sum',
                        'last_q2': 'sum',
                        'last_q3': 'sum',
                        'last_q4': 'sum',
                        'last_ytd': 'sum'})

## order vars
df_nso_all27=df_nso_all27[['metric_name','scorecard_filter','scorecard_date','last_november','last_december','last_january',
                      'last_february','last_march','last_april','last_may','last_june','last_july','last_august','last_september',
                      'last_october','last_q1','last_q2','last_q3','last_q4','last_ytd']]

######### get this year's data for national office

## get the AM data
df_metric_nso9=df_metrics1[(df_metrics1.level=='National Office') & (df_metrics1.record_date==rec_date)]


####################### CALCULATE NSO's YoY MTD AND YTD METRICS #############################################

## merge the current data to last year's data
nso_metric_calc=pd.merge(df_metric_nso9,df_nso_all27,on=['metric_name','scorecard_filter'],how="left")

## MTD,QTD,YTD calculations
yoyvarfunc(nso_metric_calc)

#### get relevant vars
df_nso_yoy=nso_metric_calc[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','Q1_YoY','q2','q2_benchmark','Q2_YoY',
                            'q3','q3_benchmark','Q3_YoY','q4','q4_benchmark','Q4_YoY','ytd','ytd_benchmark','YTD_YoY',
                            'am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]



In [ ]:
df_nso_yoy.head(5)

In [ ]:
###### concatenate the AM and SM and NSO data (excludes LEI)
df_am_yoy_metrics=pd.concat([df_am_yoy,df_sm_yoy,df_amsb_yoy, df_nso_yoy],axis=0)


In [ ]:
df_nso_yoy[(df_nso_yoy.level=='National Office') & (df_nso_yoy.metric_name=='AMSB Retail Referrals')]

In [ ]:
################################ GET GOALS FOR AMSB/AM/SM/NATIONAL OFFICE#################################
### read in goals table
goals_fil=spark.table("anp_cabbtdct1_final.SBB_SC_OUTPUT_GOALS")
goals_filter=goals_fil.toPandas()

## subset non-RAMP separately
    ##goals
goals_filter00=goals_filter[(goals_filter.sbb_region_cc!='RAMP')]
    ##data
df_am_yoy_metrics00=df_am_yoy_metrics[~df_am_yoy_metrics.am_cost_center.isin(['3838','5311']) & 
                                     ~df_am_yoy_metrics.sm_cost_center.isin(['3838','5311'])]

## subset RAMP separately
    ##goals
goals_filter01=goals_filter[(goals_filter.sbb_region_cc=='RAMP')]
    ##data
df_am_yoy_metrics01=df_am_yoy_metrics[df_am_yoy_metrics.am_cost_center.isin(['3838','5311']) | 
                                     df_am_yoy_metrics.sm_cost_center.isin(['3838','5311'])]


## merge all non-RAMP metrics with non-RAMP goals
all_goals20=pd.merge(df_am_yoy_metrics00,goals_filter00[['metric_name','amsb_logon_id','sbb_district','sbb_region_cc','level','scorecard_filter',
                     'q1_plan','q2_plan','q3_plan','q4_plan']],how="left",
                       left_on=['metric_name','acf2_id','am_cost_center','sm_cost_center','level','scorecard_filter'],
                       right_on=['metric_name','amsb_logon_id','sbb_district','sbb_region_cc','level','scorecard_filter'],
                    indicator=True)

## merge all RAMP metrics with RAMP goals
all_goals22=pd.merge(df_am_yoy_metrics01,goals_filter01[['metric_name','amsb_logon_id','level','scorecard_filter',
                     'q1_plan','q2_plan','q3_plan','q4_plan']],how="left",
                       left_on=['metric_name','acf2_id','level','scorecard_filter'],
                       right_on=['metric_name','amsb_logon_id','level','scorecard_filter'],
                    indicator=True)

##order the vars
all_goals20=all_goals20[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

all_goals22=all_goals22[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]

## concatenate RAMP and non-RAMP
all_goals2=pd.concat([all_goals22,all_goals20],axis=0)

### calculate pace to plan
q1_tot1=q1_tot.total_seconds()
q2_tot1=q2_tot.total_seconds()
q3_tot1=q3_tot.total_seconds()
q4_tot1=q4_tot.total_seconds()

q1_passed=q1_passed.total_seconds()
q2_passed=q2_passed.total_seconds()
q3_passed=q3_passed.total_seconds()
q4_passed=q4_passed.total_seconds()

## change types
#all_goals2=all_goals2=.astype({"q1": object})
#all_goals2['q1_plan']=all_goals2['q1_plan'].format(f, ".4g")
#all_goals2['q1_plan'] = all_goals2['q1_plan'].apply(Decimal)

## comment out for now
#all_goals2['q1_plan']=pd.to_numeric(all_goals2['q1_plan'], errors='coerce').fillna(0)
#all_goals2['q2_plan']=pd.to_numeric(all_goals2['q2_plan'], errors='coerce').fillna(0)
#all_goals2['q3_plan']=pd.to_numeric(all_goals2['q3_plan'], errors='coerce').fillna(0)
#all_goals2['q4_plan']=pd.to_numeric(all_goals2['q4_plan'], errors='coerce').fillna(0)
                                         

all_goals2['q1_pace_to_plan'] = np.where(all_goals2['q1_plan']>0.0,((all_goals2.q1.fillna(0))*q1_tot1/q1_passed)/(all_goals2.q1_plan), 0)
all_goals2['q2_pace_to_plan'] = np.where(all_goals2['q2_plan']>0.0,((all_goals2.q2.fillna(0))*q2_tot1/q2_passed)/(all_goals2.q2_plan), 0)
all_goals2['q3_pace_to_plan'] = np.where(all_goals2['q3_plan']>0.0,((all_goals2.q3.fillna(0))*q3_tot1/q3_passed)/(all_goals2.q3_plan), 0)
all_goals2['q4_pace_to_plan'] = np.where(all_goals2['q4_plan']>0.0,((all_goals2.q4.fillna(0))*q4_tot1/q3_passed)/(all_goals2.q4_plan), 0)


## order the vars
all_data=all_goals2[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','q1_pace_to_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','q2_pace_to_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','q3_pace_to_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','q4_pace_to_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
#all_data[(all_data.acf2_id=='TAB8991') & (all_data.metric_name=='Total Business Banking Referral Volume')]


In [ ]:
all_data.level.unique()

In [ ]:
################################### CALCULATE AMSB LEI ############################################

####################### READ IN THE CURRENT AMSB LEI DATA #######################################
df_metrics_lei= spark.table('anp_cabbtdct1_final.SBB_SC_OUTPUT_LEI')
df_metrics1_lei=df_metrics_lei.toPandas()

## get the AMSB data
lei_metric_amsb=df_metrics1_lei[(df_metrics1_lei.level=='AMSB') & (df_metrics1_lei.record_date==rec_date)]

####################### CALCULATE AM's YoY MTD AND YTD METRICS #############################################

## create prev MTD and YTD calculations
lei_metric_amsb['prev_november']=0
lei_metric_amsb['prev_december']=0
lei_metric_amsb['prev_january']=0
lei_metric_amsb['prev_february']=0
lei_metric_amsb['prev_march']=0
lei_metric_amsb['prev_april']=0
lei_metric_amsb['prev_may']=0
lei_metric_amsb['prev_june']=0
lei_metric_amsb['prev_july']=0
lei_metric_amsb['prev_august']=0
lei_metric_amsb['prev_september']=0
lei_metric_amsb['prev_october']=0
lei_metric_amsb['Q1_YoY']=0
lei_metric_amsb['Q2_YoY']=0
lei_metric_amsb['Q3_YoY']=0
lei_metric_amsb['Q4_YoY']=0
lei_metric_amsb['YTD_YoY']=0

#### get relevant vars
lei_amsb_yoy=lei_metric_amsb[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','q1_pace_to_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','q2_pace_to_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','q3_pace_to_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','q4_pace_to_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
################################### CALCULATE AM LEI ############################################

## get the AM data
lei_metric_am=df_metrics1_lei[(df_metrics1_lei.level=='AM') & (df_metrics1_lei.record_date==rec_date)]

####################### CALCULATE AM's YoY MTD AND YTD METRICS #############################################

## merge the current data to historical data
am_metric_calc_lei=pd.merge(lei_metric_am,dfamhist2,on=['metric_name','scorecard_filter','am_cost_center'],how="left")

## MTD,QTD,YTD calculations
yoyvarfunc(am_metric_calc_lei)

#### get relevant vars
lei_am_yoy=am_metric_calc_lei[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','q1_pace_to_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','q2_pace_to_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','q3_pace_to_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','q4_pace_to_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
lei_am_yoy.head(5)

In [ ]:
################################### CALCULATE SM LEI ############################################

####################### READ IN THE CURRENT SM LEI DATA #######################################

## get the SM data
lei_metric_sm=df_metrics1_lei[(df_metrics1_lei.level=='SM') & (df_metrics1_lei.record_date==rec_date)]

####################### CALCULATE SM's YoY MTD AND YTD METRICS #############################################

## merge the current data to last year's data
sm_metric_calc_lei=pd.merge(lei_metric_sm,dfsmhist2,on=['metric_name','scorecard_filter','sm_cost_center'],how="left")

## MTD,QTD,YTD calculations
yoyvarfunc(sm_metric_calc_lei)

#### get relevant vars
lei_sm_yoy=sm_metric_calc_lei[['metric_name','level','acf2_id','employee_name',
                            'november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','q1_pace_to_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','q2_pace_to_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','q3_pace_to_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','q4_pace_to_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY','am_cost_center','am_cost_center_name','am_cost_center_full_name',
                            'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
###### concatenate the AM and SM LEI data
lei_yoy_metrics=pd.concat([lei_amsb_yoy,lei_am_yoy,lei_sm_yoy],axis=0)


In [ ]:
####### if LEI exists in LEI dataframe, then delete from main dataframe
if lei_yoy_metrics.shape[0]>0:
    all_data=all_data[(all_data.metric_name!='LEI')]
else:
    print('LEI data not exist')

In [ ]:
#all_data.metric_name.unique()
#all_data[(all_data.level=="National Office") & (all_data.metric_name=="AMSB Retail Referrals")]

In [ ]:
########################################## CONCATENATE ALL METRICS WITH LEI METRIC ################################################
all_metrics_final=pd.concat([all_data,lei_yoy_metrics],axis=0)


In [ ]:
all_metrics_final.level.unique()

In [ ]:
#### define record_date
all_metrics_final['record_date']=dteq

In [ ]:
all_metrics_final.record_date.unique()

In [ ]:
#result_sm_all[(result_sm_all.sm_cost_center=='3069') & (result_sm_all.metric_name=='SBB Gross LON Volume')]


In [ ]:
########################## get NATIONAL OFFICE data ################################

df_nso_all90=all_metrics_final[(all_metrics_final.level=='National Office')]


In [ ]:
#all_metrics_final.info()

In [ ]:
all_metrics_final[(all_metrics_final.acf2_id=='ALEXAT2') & (all_metrics_final.metric_name=='Total Business Banking Referral Volume')]

In [ ]:
################ FILL BLANKS WITH 0s
all_metrics_final=all_metrics_final.replace([np.inf, -np.inf], np.nan)

all_metrics_final[['november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','q1_pace_to_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','q2_pace_to_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','q3_pace_to_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','q4_pace_to_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY']]=all_metrics_final[['november','prev_november','december','prev_december','january','prev_january',
                            'february','prev_february','march','prev_march','april','prev_april','may','prev_may',
                            'june','prev_june','july','prev_july','august','prev_august','september','prev_september',
                            'october','prev_october','q1','q1_benchmark','q1_plan','q1_pace_to_plan','Q1_YoY',
                            'q2','q2_benchmark','q2_plan','q2_pace_to_plan','Q2_YoY',
                            'q3','q3_benchmark','q3_plan','q3_pace_to_plan','Q3_YoY',
                            'q4','q4_benchmark','q4_plan','q4_pace_to_plan','Q4_YoY',
                            'ytd','ytd_benchmark','YTD_YoY']].fillna(value=0)


In [ ]:
all_metrics_final[(all_metrics_final.level=="National Office") & (all_metrics_final.metric_name=="AMSB Retail Referrals")].head(5)

In [ ]:
### Remove Puneet Sharma
dlist = ["Josie Fierro","Matthew Joseph Micallef", "Puneet Sharma","Quang Loc Huynh","Sarah Rowe","Scott Ranson","Thomas Adesunloye","Ajay Boparai"
         ,"Christian Ebolue","Neville Johnson","Shamsuddin Nasruddin Rupani","Kristyn Nazareth", 
         "Vasilios B. Chortogiannos","Oksana Oliynyk","Gaurav Jindal","Muhammad Salman Azeem", "Ipsita B Kappal","Andrew Emidio D'Alfonso","Jay Kamlesh Unadkat"
        ,"Petros Michael Zewde","George Apostolakos","Cindy Chen","Queenie Cheng","Eric Hsin Hao Tsao","John Curtin",
        "Jie Xu","Rina Tusha","Abhinav Garg","Muhammad Haroon Aslam","Moushami Dullabh","Aigbo Imoukhuede",
         "Mark Nicholas Triemstra","Sherry Conlin","Silvana S Baccega","Julie Huong Vu","Christine Caroline Evelyn Maud Sealey",
          "Erminia Spano","Suzy L Gray","Mamadou Mboup","Marie-Eve Cayer","Thomas G Ford","Tammy Lynn Demings",
          "Babar Iqbal","Rafael Garcia Zambrano","Anton Sopov","Lisa M Stecko","Trevor Nugent","Farhad Charania",
          "Manish Bhasker","Sadaf Amjad","Aaron Raymond LeBlanc","Jessie Leveille","Nancy Agnes Merrett",
          "Carrey Brown","Latanya Monteith-Housen","Stephen Kristopher Milosh","Nabeel Nawaz","Thomas Ronald Sinclair",
          "Janet Ingram","David Yang","Sherry Shi","Raman Heir","Chander Parkash Anand","Donna D Ko",
          "Rhoda Kadri","David Barth","Debra Cleal Udell","Hollie Kereszti","Didier Saenz","Angus Yeung","Maria Teixeira"]
temp = all_metrics_final[(all_metrics_final.employee_name.isin(dlist))&(all_metrics_final.record_date==dteq)].index
all_metrics_final.drop(temp , inplace=True)

In [ ]:
dlist1 = ["Maria Teixeira"] 

all_metrics_final[(all_metrics_final.employee_name.isin(dlist1))]

In [ ]:
all_metrics_final.shape

In [ ]:
all_metrics_final=all_metrics_final.fillna(0)
all_metrics_final.shape

In [ ]:
## Code for adding missing metrices
## STEP 1
metrics = ['ALL Total Commercial Volume', 'All Commercial Deposit Volume',
       'All Commercial Loan Volume','TD Securities Direct Trade','Total MUR Mortgage Volume']
base = set(metrics)

AMSBs = all_metrics_final[(all_metrics_final.level=='AMSB')][['metric_name', 'level', 'acf2_id', 'employee_name','am_cost_center', 'am_cost_center_name',
       'am_cost_center_full_name', 'sm_cost_center', 'sm_cost_center_name', 'scorecard_filter']]

d_add = pd.DataFrame(columns=['metric_name', 'level', 'acf2_id'])

for AMSB in AMSBs.acf2_id.unique():
    metric=set(AMSBs[(AMSBs.acf2_id==AMSB)].metric_name.unique())
    to_add = list(base-metric)
    for i in to_add:
        data_tem={'metric_name':[i],'level':['AMSB'], 'acf2_id':[AMSB]}
        df_temp = pd.DataFrame(data=data_tem)
        d_add = pd.concat([d_add, df_temp], ignore_index=True)
        
AMSB1 = (AMSBs.drop(['metric_name','level'], axis=1)).drop_duplicates()
AMSB1.head(5)

In [ ]:
# STEP 2
prelim = pd.merge(d_add,AMSB1, how='left', on=['acf2_id'])
colls = ['november', 'prev_november', 'december', 'prev_december', 'january', 'prev_january',
       'february', 'prev_february', 'march', 'prev_march', 'april', 'prev_april', 'may', 'prev_may', 'june', 'prev_june', 'july', 'prev_july',
       'august', 'prev_august', 'september', 'prev_september', 'october', 'prev_october', 'q1', 'q1_benchmark', 'q1_plan', 'q1_pace_to_plan',
       'Q1_YoY', 'q2', 'q2_benchmark', 'q2_plan', 'q2_pace_to_plan', 'Q2_YoY', 'q3', 'q3_benchmark', 'q3_plan', 'q3_pace_to_plan', 'Q3_YoY', 'q4',
       'q4_benchmark', 'q4_plan', 'q4_pace_to_plan', 'Q4_YoY', 'ytd', 'ytd_benchmark', 'YTD_YoY']
for i in colls:
    prelim[i]=0

prelim['record_date']=dteq 

Final = prelim[['metric_name', 'level', 'acf2_id', 'employee_name', 'november', 'prev_november', 'december', 'prev_december', 'january', 'prev_january',
       'february', 'prev_february', 'march', 'prev_march', 'april', 'prev_april', 'may', 'prev_may', 'june', 'prev_june', 'july', 'prev_july',
       'august', 'prev_august', 'september', 'prev_september', 'october', 'prev_october', 'q1', 'q1_benchmark', 'q1_plan', 'q1_pace_to_plan',
       'Q1_YoY', 'q2', 'q2_benchmark', 'q2_plan', 'q2_pace_to_plan', 'Q2_YoY', 'q3', 'q3_benchmark', 'q3_plan', 'q3_pace_to_plan', 'Q3_YoY', 'q4',
       'q4_benchmark', 'q4_plan', 'q4_pace_to_plan', 'Q4_YoY', 'ytd', 'ytd_benchmark', 'YTD_YoY', 'am_cost_center', 'am_cost_center_name',
       'am_cost_center_full_name', 'sm_cost_center', 'sm_cost_center_name', 'scorecard_filter','record_date']]

#Final.head(2)
frames = [Final, all_metrics_final]
all_metrics_final = pd.concat(frames)

In [ ]:
all_metrics_final.shape

In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table; this dataframe contains all metrics except for LEI
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(all_metrics_final).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_DASHBOARD_F24")


In [ ]:
### END OF PROGRAM
all_metrics_final.head(5)
#all_metrics_final.metric_name.unique()

In [ ]:
Hist_Cols = ['november', 'prev_november', 'december', 'prev_december', 'january', 'prev_january',
             'february', 'prev_february', 'march', 'prev_march', 'april', 'prev_april','may', 'prev_may', 'june', 'prev_june', 'july', 'prev_july',
       'august', 'prev_august', 'september', 'prev_september', 'october', 'prev_october','q1', 'q1_benchmark', 'q1_plan', 'q1_pace_to_plan',
            'Q1_YoY', 'q2', 'q2_benchmark', 'q2_plan', 'q2_pace_to_plan', 'Q2_YoY','q3', 'q3_benchmark', 'q3_plan', 'q3_pace_to_plan', 'Q3_YoY', 'q4',
       'q4_benchmark', 'q4_plan', 'q4_pace_to_plan', 'Q4_YoY', 'ytd', 'ytd_benchmark', 'YTD_YoY']



In [ ]:
record_date = dteq
record_date

In [ ]:
#spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
#spark.createDataFrame(df_Hist).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_DASHBOARD_F24")


In [ ]:
df_Hist.metric_name.unique()

In [ ]:
all_metrics_final.metric_name.unique()